In [1]:
!bash -c 'source activate tl-detect && conda install -qy -c conda-forge pyyaml'

Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda/envs/tl-detect

  added / updated specs: 
    - pyyaml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pyyaml-3.12                |           py35_1         394 KB  conda-forge
    yaml-0.1.7                 |                0         302 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         696 KB

The following NEW packages will be INSTALLED:

    pyyaml: 3.12-py35_1 conda-forge
    yaml:   0.1.7-0     conda-forge

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [2]:
import yaml
import pandas as pd
import cv2
import os
import pathlib
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from io import BytesIO
import PIL.Image

# See https://gist.github.com/kylemcdonald/2f1b9a255993bf9b2629
def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [3]:
# This is needed to display the images.
%matplotlib inline

In [4]:
def load_bosch(bosch_path, file):
    with open(os.path.join(bosch_path, file), 'r') as f:
        df = pd.io.json.json_normalize(yaml.load(f))
    boxes_len = df['boxes'].apply(lambda x: len(x))
    return df[boxes_len > 2]

In [6]:
def extract_tl(boxes, img_path, bosch_path, output_dir):
    src_dir, src_name = os.path.split(img_path)
    img = cv2.imread(os.path.join(bosch_path, img_path))
    if img is None:
        raise ValueError('for file: ' + os.path.join(bosch_path, img_path) + ' found img type:' + str(type(img)))

    for i, b in enumerate(boxes):
        if not b['occluded']:
            x_min = np.int(boxes[1]['x_min']-3)
            x_max = np.int(boxes[1]['x_max']+3)
            y_min = np.int(boxes[1]['y_min']-3)
            y_max = np.int(boxes[1]['y_max']+3)
            label = b['label']
            dst_dir = os.path.join(output_dir, label)

            if not os.path.exists(dst_dir):
                pathlib.Path(dst_dir).mkdir(parents=True)

            fn = src_name + str(i) + ".jpg"
            cv2.imwrite(os.path.join(dst_dir, fn), img[y_min:y_max,x_min:x_max])

In [7]:
img_path = 'imgs/bosch/train'
data = load_bosch(img_path, 'train.yaml')
data.apply(lambda x: extract_tl(boxes=x[0], img_path=x[1], 
                                bosch_path=img_path,
                                output_dir=os.path.join('imgs', 'training', 'bosch')), axis=1);

In [8]:
greenright = cv2.imread('imgs/training/bosch/GreenRight/10648.png0.jpg')

showarray(greenright[..., ::-1])